In [1]:
from germanhass.DBCode.HassDBAdapter import HassDBAdapter
from tqdm import tqdm
from multiprocess import Pool
import pandas as pd
from os.path import join
import numpy as np

/home/jana/anaconda3/envs/ri/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# connect to the db
db = HassDBAdapter()

In [3]:
def extract_information(entry):
    row = {
        'id':entry['_id'],
        'text':' '.join(entry['tokens']),
        'label':entry['account_type']
    }
    return row

In [4]:
dst = '../../data/traindata'
number_of_cores = 10
pool = Pool(number_of_cores)

## Dataset 1

In [19]:
test_hass = list(db.hate_tweets_coll\
    .find({"test_set":"dataset1"}))
train_hass = list(db.hate_tweets_coll\
    .find({"training_set":"dataset1"}))

test_counter = list(db.counter_tweets_coll\
    .find({"test_set":"dataset1"}))
train_counter = list(db.counter_tweets_coll\
    .find({"training_set":"dataset1"}))

In [22]:
len(test_hass)

50000

In [23]:
train = pd.DataFrame()
tweets = train_hass + train_counter
for row in tqdm(pool.imap_unordered(func=extract_information, 
                    iterable=tweets), total=len(tweets)):
        train = train.append(row, ignore_index=True)
#train.to_csv(join(dst, 'train_sample.csv.gzip'), index=False, compression='gzip')

100%|██████████████████████████████████████████████████████| 1000000/1000000 [10:47:27<00:00, 25.74it/s]


In [24]:
test = pd.DataFrame()
tweets = test_hass[0:-10000] + test_counter[0:-10000]
for row in tqdm(pool.imap_unordered(func=extract_information, 
                    iterable=tweets), total=len(tweets)):
        test = test.append(row, ignore_index=True)
#test.to_csv(join(dst, 'test_sample.csv.gzip'), index=False, compression='gzip')

100%|████████████████████████████████████████████████████████████| 80000/80000 [06:39<00:00, 200.50it/s]


In [25]:
val = pd.DataFrame()
tweets = test_hass[-10000:] + test_counter[-10000:]
for row in tqdm(pool.imap_unordered(func=extract_information, 
                    iterable=tweets), total=len(tweets)):
        val = val.append(row, ignore_index=True)
#val.to_csv(join(dst, 'val_sample.csv.gzip'), index=False, compression='gzip')

100%|████████████████████████████████████████████████████████████| 20000/20000 [00:54<00:00, 364.07it/s]


In [ ]:
for df, fname in zip([train, test, val], ['train', 'test', 'val']):
    df['label'] = df['label'].replace({'counter':0, 'hate':1})
    df = df.sample(frac=1, random_state=1).reset_index(drop=True)
    df['text'] = df['text'].apply(lambda x: x.replace('\n', ' '))
    np.savetxt(join(dst, '{}_text.txt'.format(fname)), df['text'].values, fmt='%s')
    np.savetxt(join(dst, '{}_labels.txt'.format(fname)), df['label'].values, fmt='%s')

## Dataset DE

In [5]:
test_hass = list(db.hate_tweets_coll\
    .find({"test_set":"datasetDE"}))
train_hass = list(db.hate_tweets_coll\
    .find({"training_set":"datasetDE"}))

test_counter = list(db.counter_tweets_coll\
    .find({"test_set":"datasetDE"}))
train_counter = list(db.counter_tweets_coll\
    .find({"training_set":"datasetDE"}))

In [6]:
train = pd.DataFrame()
tweets = train_hass[0:25000] + train_counter[0:25000]
for row in tqdm(pool.imap_unordered(func=extract_information, 
                    iterable=tweets), total=len(tweets)):
        train = train.append(row, ignore_index=True)
train.to_csv(join(dst, 'dataset_DE_sample_train.csv'), index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [04:42<00:00, 176.75it/s]


In [ ]:
train = pd.DataFrame()
tweets = train_hass + train_counter
for row in tqdm(pool.imap_unordered(func=extract_information, 
                    iterable=tweets), total=len(tweets)):
        train = train.append(row, ignore_index=True)
train.to_csv(join(dst, 'dataset_DE_train.csv'), index=False)

 39%|███████████████████████████████████████████████████████████████                                                                                                  | 391895/1000000 [2:28:27<5:30:56, 30.63it/s]